In [1]:
import arcpy
import pandas as pd
import os
import wfrc_bike_lanes as wfrc

In [1]:
wfrc_bike_lanes = r'C:\giswork\multimodal\bikelanes\SourceData.gdb\Bike_Network_WFRC'
existing_status_field = 'Status'
network_field = 'Network'
lane_type_field = 'Type'

roads = r'C:\Users\kwalker\Documents\ArcGIS\Projects\MultiModal bike lanes\UTRANS.agrc.utah.gov.sde\UTRANS.TRANSADMIN.Roads_Edit'

working_dir = r'C:\giswork\multimodal\bikelanes'
working_data_gdb_name = r'WorkingData.gdb'
working_data_gdb = os.path.join(working_dir, working_data_gdb_name)

if not os.path.exists(working_dir):
    os.mkdir(working_dir)

if arcpy.Exists(working_data_gdb):
    arcpy.management.Delete(working_data_gdb)

arcpy.management.CreateFileGDB(working_dir, working_data_gdb_name)

<Result 'C:\\giswork\\multimodal\\bikelanes\\WorkingData.gdb'>

In [2]:
# Select Regional and exitisting bike lanes
if arcpy.Exists('existing_regional'):
    arcpy.management.Delete('existing_regional')
    
existing_regional_lanes = arcpy.management.MakeFeatureLayer(
wfrc_bike_lanes,
'existing_regional',
"""{} = 'Regional' And {} = 'Existing'""".format(network_field, existing_status_field))[0]

In [3]:
# Buffer bike lanes and merge buffer by Status and Type
bikelane_buffer_dist = 4
merge_fields = [existing_status_field, lane_type_field]
wfrc_bike_buffer = wfrc.buffer_bike_lanes(existing_regional_lanes, merge_fields, working_data_gdb, bikelane_buffer_dist)

In [4]:
# Find all roads within a bike lane buffer
wrfc_buffer_roads_join = wfrc.join_roads_to_bike_buffer(roads, wfrc_bike_buffer, working_data_gdb)

In [5]:
dataframe = wfrc.get_unique_lane_types(wrfc_buffer_roads_join, lane_type_field, working_data_gdb)

In [6]:
dataframe

([],                   Type
 0       Bike Boulevard
 1            Bike Lane
 2   Buffered Bike Lane
 3  On-street Connector
 4  Protected Bike Lane
 5          Shared Lane
 6      Shared Use Path
 7     Shoulder Bikeway
 8                Trail)

In [9]:
dataframe[1]

,Type
0,Bike Boulevard
1,Bike Lane
2,Buffered Bike Lane
3,On-street Connector
4,Protected Bike Lane
5,Shared Lane
6,Shared Use Path
7,Shoulder Bikeway
8,Trail


In [21]:
joinframe = None
with arcpy.da.SearchCursor(wrfc_buffer_roads_join,
                           ['STATUS', 'BIKE_L', 'BIKE_R', 'BIKE_PLN_L', 'BIKE_PLN_R', 'Status_1', 'Type']) as cursor:
    joinframe = wfrc.arcgis_to_pandas(cursor)
joinframe.to_pickle('./joinframe.pkl')

NameError: name 'wrfc_buffer_roads_join' is not defined

## Restart the notebook here to load from pickled dataframe

In [3]:
joinframe = pd.read_pickle('./joinframe.pkl')
joinframe.head(10)

,STATUS,BIKE_L,BIKE_R,BIKE_PLN_L,BIKE_PLN_R,Status_1,Type
0,,2B,2B,E,E,Existing,Shoulder Bikeway
1,,2B,2B,E,E,Existing,Shoulder Bikeway
2,,2B,2B,E,E,Existing,Bike Lane
3,,2B,2B,E,E,Existing,Bike Lane
4,,2B,2B,E,E,Existing,Shoulder Bikeway
5,,2B,2B,E,E,Existing,Shoulder Bikeway
6,,2B,2B,P,P,Existing,Shared Lane
7,,2B,2B,E,E,Existing,Shoulder Bikeway
8,,2B,2B,E,E,Existing,Bike Lane
9,,2B,2B,E,E,Existing,Bike Lane


## Count unique values in *Roads* fields for all *WFRC* bike lane Types

In [20]:
typegroup = joinframe.groupby('Type')
typegroup.nunique()

,STATUS,BIKE_L,BIKE_R,BIKE_PLN_L,BIKE_PLN_R,Status_1,Type
Type,,,,,,,
Bike Boulevard,1,2,2,1,1,1,1
Bike Lane,3,6,6,3,3,1,1
Buffered Bike Lane,1,3,3,3,3,1,1
On-street Connector,1,1,1,1,1,1,1
Protected Bike Lane,1,2,2,2,2,1,1
Shared Lane,1,3,3,3,3,1,1
Shared Use Path,2,5,5,3,3,1,1
Shoulder Bikeway,1,4,4,3,3,1,1
Trail,1,1,1,2,2,1,1


In [34]:
bikel_counts = joinframe.groupby('Type')['BIKE_L'].value_counts()

In [35]:
bikel_counts

Type                 BIKE_L
Bike Boulevard       3C          10
                     2B           2
Bike Lane            2B        1963
                                196
                     3           12
                     3B          10
                     3C           6
                     1            1
Buffered Bike Lane   2B          64
                                  7
                     3C           1
On-street Connector               4
Protected Bike Lane  2B          20
                     3C           3
Shared Lane          3B         615
                     2B          83
                                 27
Shared Use Path      2B         146
                                 16
                     3B           4
                     2C           2
                     3C           1
Shoulder Bikeway     2B         470
                                 21
                     3C           4
                     3B           2
Trail                            15


In [37]:
unique_bikel = typegroup.get_group('Bike Boulevard')['BIKE_L']

In [38]:
unique_bikel

2009    3C
2010    3C
2011    3C
2012    3C
2013    3C
2014    2B
2015    3C
2016    3C
2017    2B
2018    3C
2019    3C
2023    3C
Name: BIKE_L, dtype: object